1. Setting Up the Environment and Dependencies

In [ ]:
!pip install super-gradients torch torchvision pycocotools


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.5/684.5 kB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of albumentations to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of albumentations to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.3/108.3 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2. import necessary modules

In [ ]:
import os
import torch
from super_gradients.training import models, Trainer
from super_gradients.training import Trainer, dataloaders, models
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train,
    coco_detection_yolo_format_val
)
from super_gradients.training.models.detection_models.pp_yolo_e import (
    PPYoloEPostPredictionCallback
)
#for data visualization
import matplotlib.pyplot as plt
import cv2

[2024-11-12 01:06:49] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /root/sg_logs/console.log


In [ ]:
# Paths to dataset directories in Google Drive
data_dir = '/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco'
train_img_dir = os.path.join(data_dir, "train")
val_img_dir = os.path.join(data_dir, "valid")
test_img_dir = os.path.join(data_dir, "test")
# Dataset classes
CLASSES = ['free_parking_space', 'not_free_parking_space', 'partially_free_parking_space']
NUM_CLASSES = len(CLASSES)

# Dataloader parameters
DATALOADER_PARAMS = {
    'batch_size': 8,
    'num_workers': 2
}
# Paths to dataset directories in Google Drive
data_dir = '/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco'
train_img_dir = os.path.join(data_dir, "train")
val_img_dir = os.path.join(data_dir, "valid")
test_img_dir = os.path.join(data_dir, "test")

In [ ]:
class Config:
    CHECKPOINT_DIR = '/content/drive/MyDrive/College car parking space-20241109T021439Z-001/checkpoints'
    EXPERIMENT_NAME = 'race_number' #specify the experiment name
    #dataset params
    DATA_DIR = '/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco'
    TRAIN_IMAGES_DIR = '/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/train/images'
    TRAIN_LABELS_DIR = '/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/train/labels'
    VAL_IMAGES_DIR = '/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/valid/images'
    VAL_LABELS_DIR = '/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/valid/labels'
    TEST_IMAGES_DIR = '/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/test/images'
    TEST_LABELS_DIR = '/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/test/labels'
    CLASSES = ['free_parking_space', 'not_free_parking_space', 'partially_free_parking_space','person']
    NUM_CLASSES = len(CLASSES)
    DATALOADER_PARAMS = {
        'batch_size': 8,
        'num_workers': 2
    }
config = Config()

3.generating individuals labels for the coco model

In [ ]:
import json
import os

def convert_coco_to_yolo(coco_json_path, output_dir):
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)

    # Create directories if they don't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    images = {image['id']: image for image in coco_data['images']}
    annotations = coco_data['annotations']

    for ann in annotations:
        image_id = ann['image_id']
        image_info = images[image_id]
        image_filename = os.path.splitext(image_info['file_name'])[0]
        label_filename = os.path.join(output_dir, image_filename + '.txt')

        # YOLO format: <object-class> <x_center> <y_center> <width> <height>
        x_center = (ann['bbox'][0] + ann['bbox'][2] / 2) / image_info['width']
        y_center = (ann['bbox'][1] + ann['bbox'][3] / 2) / image_info['height']
        width = ann['bbox'][2] / image_info['width']
        height = ann['bbox'][3] / image_info['height']

        # Assuming 'category_id' starts from 0 and matches the indices in config.CLASSES
        category_id = ann['category_id'] - 1

        with open(label_filename, 'a') as f:
            f.write(f"{category_id} {x_center} {y_center} {width} {height}\n")

# Convert train annotations
convert_coco_to_yolo('/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/train/_annotations.coco.json','/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/train/labels')

# Convert validation annotations
convert_coco_to_yolo('/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/valid/_annotations.coco.json','/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/valid/labels')

# Convert test annotations
convert_coco_to_yolo('/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/test/_annotations.coco.json','/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/test/labels')


In [ ]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.TRAIN_IMAGES_DIR,
        'labels_dir': config.TRAIN_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.VAL_IMAGES_DIR,
        'labels_dir': config.VAL_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': config.DATA_DIR,
        'images_dir': config.TEST_IMAGES_DIR,
        'labels_dir': config.TEST_LABELS_DIR,
        'classes': config.CLASSES
    },
    dataloader_params=config.DATALOADER_PARAMS
)

[2024-11-12 01:26:21] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 140/140 [00:01<00:00, 113.10it/s]
[2024-11-12 01:26:23] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations: 100%|██████████| 40/40 [00:00<00:00, 149.61it/s]
[2024-11-12 01:26:23] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.


In [ ]:
train_data.dataset.plot()

In [ ]:
test_data.dataset.plot()

In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class ParkingSpaceDataset(Dataset):
    def __init__(self, images_dir, labels_dir, classes, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.classes = classes
        self.transform = transform
        self.image_paths = [os.path.join(images_dir, img) for img in os.listdir(images_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]
        self.label_paths = [os.path.join(labels_dir, label) for label in os.listdir(labels_dir) if label.endswith('.txt')]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label_path = self.label_paths[idx]

        # Load image
        image = Image.open(img_path).convert("RGB")

        # Load and parse label
        with open(label_path, 'r') as f:
            labels = f.readlines()

        # Extract class indices and bounding box coordinates
        targets = []
        for label in labels:
            parts = label.strip().split()
            class_idx = int(parts[0])
            bbox = list(map(float, parts[1:]))
            targets.append([class_idx] + bbox)  # You can change this format as per your requirement

        # Convert targets to tensor
        targets = torch.tensor(targets)

        # Apply transformations
        if self.transform:
            image = self.transform(image)

        return image, targets

# Define the transforms
transform = transforms.Compose([
    transforms.Resize((416, 416)),
    transforms.ToTensor()
])

# Create dataset instances with the transformation applied
train_dataset = ParkingSpaceDataset(
    images_dir=os.path.join(config.DATA_DIR, config.TRAIN_IMAGES_DIR),
    labels_dir=os.path.join(config.DATA_DIR, config.TRAIN_LABELS_DIR),
    classes=config.CLASSES,
    transform=transform
)

val_dataset = ParkingSpaceDataset(
    images_dir=os.path.join(config.DATA_DIR, config.VAL_IMAGES_DIR),
    labels_dir=os.path.join(config.DATA_DIR, config.VAL_LABELS_DIR),
    classes=config.CLASSES,
    transform=transform
)

test_dataset = ParkingSpaceDataset(
    images_dir=os.path.join(config.DATA_DIR, config.TEST_IMAGES_DIR),
    labels_dir=os.path.join(config.DATA_DIR, config.TEST_LABELS_DIR),
    classes=config.CLASSES,
    transform=transform
)

# Custom collate function to handle varying sizes or types
def custom_collate_fn(batch):
    images, labels = zip(*batch)
    images = torch.stack(images, 0)
    return images, labels  # Note: Labels might not need stacking depending on format

# Create DataLoader instances
train_loader = DataLoader(train_dataset, batch_size=config.DATALOADER_PARAMS['batch_size'], shuffle=True, num_workers=config.DATALOADER_PARAMS['num_workers'], collate_fn=custom_collate_fn)
val_loader = DataLoader(val_dataset, batch_size=config.DATALOADER_PARAMS['batch_size'], shuffle=False, num_workers=config.DATALOADER_PARAMS['num_workers'], collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=config.DATALOADER_PARAMS['batch_size'], shuffle=False, num_workers=config.DATALOADER_PARAMS['num_workers'], collate_fn=custom_collate_fn)


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 17045, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 17045 (delta 10), reused 18 (delta 8), pack-reused 17022 (from 1)
Receiving objects: 100% (17045/17045), 15.67 MiB | 16.44 MiB/s, done.
Resolving deltas: 100% (11705/11705), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 76.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
super-gradients 3.7.1 requires numpy<=1.23, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import yaml

# Define the data configuration
data_config = {
    'train': '/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/train',
    'val': '/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/valid',
    'test': '/content/drive/MyDrive/College car parking space-20241109T021439Z-001/car parking space.v1i.coco/test',
    'nc': 4,
    'names': ['free_parking_space', 'not_free_parking_space', 'partially_free_parking_space', 'person']
}

# Save the configuration to a YAML file
data_yaml = '/content/data.yaml'
with open(data_yaml, 'w') as file:
    yaml.dump(data_config, file)

print(f"Data configuration saved to {data_yaml}")


Data configuration saved to /content/data.yaml


In [ ]:
import os
from pathlib import Path

# Set the path to the YOLOv5 repository
yolov5_repo_path = Path('/content/yolov5')

# Change the working directory to the YOLOv5 repository
%cd {yolov5_repo_path}

# Train the YOLOv5 model using the custom dataset
!python train.py --img 640 --batch 8 --epochs 10 --data {data_yaml} --weights yolov5s.pt --cache


/content/yolov5
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-11-12 01:27:01.871662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 01:27:01.928394: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 01:27:01.945144: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: 

In [ ]:
import torch
from pathlib import Path

# Set the path to the YOLOv5 repository
yolov5_repo_path = Path('/content/yolov5')

# Change the working directory to the YOLOv5 repository
%cd {yolov5_repo_path}

# Load the pre-trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)  # or use your own model

# Modify the final detection layer to match the number of classes in your dataset
num_classes = 4  # Your number of classes
model.nc = num_classes  # Set the number of classes to match your dataset

# Check the model summary
print(model)

# Train the model using your custom dataset
!python train.py --img 640 --batch 8 --epochs 10 --data {data_yaml} --weights 'yolov5s.pt' --cache


/content/yolov5


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-11-12 Python-3.10.12 torch-2.5.0+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [ ]:
# Save the modified model if needed (optional)
torch.save(model.state_dict(), 'yolov5s_modified1.pt')

In [ ]:
# Load the saved YOLOv5 model
model = torch.load('/content/yolov5/yolov5s_modified1.pt')  # Assuming this is the path to your model

model.eval()
# Load the image for inference
img = Image.open('/content/1.png')  # Path to your image

# Perform inference
results = model(img)

# Display results with bounding boxes and class labels
results.show()